In [1]:
import pandas as pd 
import numpy  as np
import json

file_path = '/home/mantovani/Documents/IC/HMC_TE-s/Temp_Data/Dados_Dominios_Conservados/sequences.json'

df:dict = {'Name': []}

with open(file_path, mode='r') as file:

    dados = json.load(file)

    for i in dados['results']:

        df['Name'].append(i['crossReferences'][0]['name'].strip()[0:11])

        l_reading_frames, l_df = len(i['openReadingFrames']), (len(df)-1)/4

        for j in range(max(l_reading_frames, l_df)):

            reading_frames = i['openReadingFrames'][j]

            if j > l_df:

                l = [np.nan for i in range(len(df['Name'])-1)]#pois o nome é gerado antes

                dados['Start_{}'.format(j)]             = l
                dados['End_{}'.format(j)]               = l
                dados['Strand_{}'.format(j)]            = l
                dados['Protein_Sequence_{}'.format(j)]  = l

            elif j <= l_reading_frames:

                dados['Start_{}'.format(j)].append(reading_frames['start'])
                dados['End_{}'.format(j)].append(reading_frames['end'])
                dados['Strand_{}'.format(j)].append(reading_frames['strand'])
                dados['Protein_Sequence_{}'.format(j)].append(reading_frames['protein']['sequence'])

            else:

                dados['Start_{}'.format(j)].append(np.nan)
                dados['End_{}'.format(j)].append(np.nan)
                dados['Strand_{}'.format(j)].append(np.nan)
                dados['Protein_Sequence_{}'.format(j)].append(np.nan)

    print(dados)

    file.close()

IndentationError: expected an indented block after 'if' statement on line 25 (671681887.py, line 29)

In [5]:
import pandas as pd 
import numpy as np

meu_dict = {
    'chave1': ['subchave1', 'subvalor1', 'subchave2', 'subvalor2'],
    'chave2': ['valor2']
} 

max_len = max(len(v) for v in meu_dict.values())

# Preencher as listas menores com NaN
for key in meu_dict:
    if len(meu_dict[key]) < max_len:
        meu_dict[key].extend([np.nan] * (max_len - len(meu_dict[key])))

meu_dict = pd.DataFrame(meu_dict)

meu_dict.head()

,chave1,chave2
0,subchave1,valor2
1,subvalor1,NaN
2,subchave2,NaN
3,subvalor2,NaN


In [11]:
print([np.nan for i in range(10)])

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
